# Analysing False Positive Genuine files in dev set - version 2.0 dataset

> Get SLIME explanation for all those genuine FALSE Positive files. Meaning, these files are genuine files but it is misclassified as spoofed !! Looking at the SLIME explanation we try to infer meaning/correlation with our other observations !


## The top 10 genuine FP files

         Gen prob     Spf Prob      Log Liklihood ratio
                               
    

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image

%matplotlib inline

In [3]:
# Load the prediction file
# Predictions file that has three columns: genuin, spoofed prob and log likehood ratio

pred_file='model_3sec_relu_0.5_run8/predictions/train_prediction.txt'
#new_pred_file = 'model_3sec_relu_0.5_run8/predictions/train_prediction_with_index.txt'
new_pred_file = 'predictions/train_prediction_with_index.txt'

In [4]:
# Create a new prediction file where we write index (starting from 0) as the first column
# This way things become much easier

'''
with open(pred_file) as f, open(new_pred_file,'w') as f2:
    i=0
    for line in f:
        f2.write(str(i)+' '+ line)
        #print(i)
        i+=1    
'''

# Above code is just used to append file index in the prediction file
# Careful when re-running

"\nwith open(pred_file) as f, open(new_pred_file,'w') as f2:\n    i=0\n    for line in f:\n        f2.write(str(i)+' '+ line)\n        #print(i)\n        i+=1    \n"

In [5]:
# Separate out predictions of genuine and spoofed files
# Training set has first 1507 as genuine and remaining 1507 as spoofed

all_predictions = []
all_gens = []
all_spoofs = []

with open(new_pred_file) as f:
    all_predictions = [line.strip() for line in f]
    
all_gen_predictions = all_predictions[:1507]                 # first 1507 files are genuine
all_spf_predictions = all_predictions[1507:]                 # files from 1507 onwards are spoofed examples

In [6]:
print(len(all_gen_predictions))
print(len(all_spf_predictions))

1507
1507


In [7]:
# Collect all misclassified genuine files for which spoofed class got probability > 70%
# the strongly correctly classified cases
'''
fname='index_file_list/allGenIndexList_FP.txt'
count=0
splitIndex = 2                   # We want spoofed class probability
predList = all_gen_predictions   

#print(len(predList))

with open(fname,'w') as f2:
    for items in predList:
        prob = items.strip().split(' ')[splitIndex]   
        if float(prob) > 0.7:
            print(items)
            count+=1
            f2.write(items+'\n')            
print(count)
'''

# There is only one file with 78% probability being misclassified in training set

# Careful when you re-run this code. It is use to take files with 90% probability to do analysis using slime !

"\nfname='index_file_list/allGenIndexList_FP.txt'\ncount=0\nsplitIndex = 2                   # We want spoofed class probability\npredList = all_gen_predictions   \n\n#print(len(predList))\n\nwith open(fname,'w') as f2:\n    for items in predList:\n        prob = items.strip().split(' ')[splitIndex]   \n        if float(prob) > 0.7:\n            print(items)\n            count+=1\n            f2.write(items+'\n')            \nprint(count)\n"

In [8]:
%%bash
cat index_file_list/allGenIndexList_FP.txt

607 0.219429 0.780571 -1.269


### Total misclassified genuine files where spoofed class got > 75% probability is = 1 (out of 1507)**

## Analysing Time: The top two components from SLIME - False positive Genuine case

Note, that under time analysis, we have cut our input spectrogram into 10 different temporal components/segments, where each segment correpsonds to

> ***300 mili seconds***

In [9]:
# Show how we cut the spectrogram in timexfrequency

Image("time.png")

In [10]:
# Load the explanation file to see the distribution

file = 'top_two_explanation_indices/time/gen_FP_box.txt'

In [11]:
def get_top1_top2_list(file):
    with open(file) as f:
        top1 = [int(line.strip().split(' ')[0]) for line in f]
    with open(file) as f:  
        top2 = [int(line.strip().split(' ')[1]) for line in f]
    return top1, top2    

In [12]:
# Get top1 and top2 in seperate list
top1, top2 = get_top1_top2_list(file)

In [13]:
print(len(top1))
print(len(top2))

1
1


In [14]:
def get_counts(datalist, key):
    count=0
    for i in range(0,len(datalist)):
        if datalist[i] == key:
            count+=1
    return count            

In [15]:
def print_component_distribution(comps, predList, order):
    print('Printing component weigting distribution for Top:', order)
    for i in comps:
        print('Component ' + str(i) + ' : ' + str(get_counts(predList, i)))    

### Using the top1 components - given by SLIME (Time segmentation)
 

In [16]:
comps=[0,1,2,3,4,5,6,7,8,9]    # In time we have 10 segments/components

In [17]:
# Print distribution on top1
order = 1
print_component_distribution(comps, top1, order)

Printing component weigting distribution for Top: 1
Component 0 : 1
Component 1 : 0
Component 2 : 0
Component 3 : 0
Component 4 : 0
Component 5 : 0
Component 6 : 0
Component 7 : 0
Component 8 : 0
Component 9 : 0


   


    Component 0 : 1
    Component 1 : 0
    Component 2 : 0
    Component 3 : 0
    Component 4 : 0
    Component 5 : 0
    Component 6 : 0
    Component 7 : 0
    Component 8 : 0
    Component 9 : 0
    


** Observation**
> We see that out of these FP genuine files, the highest vote/explanation is favoured for component 0 (corresponding to 1000ms signal). ***It is similar to spoofed class behaviour***. In our spoofed class analysis we were making a hypothesis that the DTMF tones that were used for concatenating the signal is leared by the CNN for spoof class. These genuine files somehow exhibit those property and hence misclassified as spoofed (here we have taken 7 genuine files that were misclassified to check the distribution, with probability of spoofed class > 70%. We find for each of these files that highest explanation is for Component 1)

# Pick 5 audio files for hearing

In [165]:
#random_5_ids = [81,4,289,47,283]   # Genuine misclassified as spoofed with >70% probability
#add 1 to each to access correct file index in real world

#random_5_ids = [82,5,290,48,284]
#base='/homes/bc305/myphd/datasets/ASVSpoof2017_v2.0/ASVspoof2017_V2_dev/'

In [50]:
%%bash
#cp /homes/bc305/myphd/datasets/ASVSpoof2017_v2.0/ASVspoof2017_V2_dev/D_1000284.wav audio_files_hearing/genuine_FP/

#ls audio_files_hearing/genuine_FP/

## Analysing Freq: The top two components from SLIME - True positive Genuine case

Note, that under frequency analysis, we have cut our input spectrogram into 8 different frequency components/segments, where each segment correpsonds to

> ***1000 Hz frequency***

In [18]:
# Show how we cut the spectrogram in timexfrequency

Image("frequency.png")

In [19]:
# Get spectral explanation file

file = 'top_two_explanation_indices/freq/gen_FP_box.txt' # using box

In [20]:
%%bash
cat 'top_two_explanation_indices/freq/gen_FP_box.txt' | head

# the top two components 7 6 dominates the explanation

3 5


In [21]:
# Get top1 and top2 in seperate list
top1, top2 = get_top1_top2_list(file)

In [22]:
print(len(top1))
print(len(top2))

1
1


## Top 1 component distribution - Frequency

In [23]:
comps = [0,1,2,3,4,5,6,7]   # in Frequency we have 8 components

In [24]:
# Print distribution on top1

order = 1
print_component_distribution(comps, top1, order)

Printing component weigting distribution for Top: 1
Component 0 : 0
Component 1 : 0
Component 2 : 0
Component 3 : 1
Component 4 : 0
Component 5 : 0
Component 6 : 0
Component 7 : 0


> **Using new Box spectrograms**, and also we corrected the labels for explanation:

Its same like before

# Pick 5 audio files for hearing

We take the other two remaining files also

In [62]:
%%bash
#cp /homes/bc305/myphd/datasets/ASVSpoof2017_v2.0/ASVspoof2017_V2_dev/D_1000012.wav audio_files_hearing/genuine_FP/

#ls audio_files_hearing/genuine_FP/

** Observations**

> We find that all these 17 genuine files were misclassified as spoofed because these files exhibit the same property in high frequency regions that our spoofed class has learned. The frequency component 8 corresponding to 7000-8000 Hz, seems to get maximally activated for making the prediction. 

 * This does not strongly corr-relate to our classes though. Our genuine class showed highest vote for component 7 and spoofed showed component 6. But, nevertheless, its shows that high frequency regions relate to class prediction



# Overall Summary from this study

We found 17 genuine audio files that were misclassified to be spoofed (with probability > 50%). Here we present brief summary under each methods.

**Time segmentation**

> The highest vote/explanation is favoured for component 3 (corresponding to 9000-1200ms signal). It is similar to genuine class behaviour, but we were expecting it to show properties that we found with spoofed class explanations. Note that spoofed class, under time analysis, give high weightage to first 300ms (which points to the fact that the DTMF tone is still causing the influence in spoofed class prediction !)

**Frequency segmentation**

> We find that all these 17 genuine files were misclassified as spoofed because these files exhibit the same property in high frequency regions that our spoofed class has learned. The frequency component 8 corresponding to 7000-8000 Hz, seems to get maximally activated for making the prediction. 

 

# Observations


> In the training set, **we find 1 spoofed files that has been misclassified as genuine with more than 75% ** probability by the CNN end-to-end model (no file exist with 90% prob). We take this file and explore explanations from SLIME across time and frequency. This should exactly match with the class behaviour we found earlier for each classes.

**Temporal**

    Component 0 : 1
    Component 1 : 0
    Component 2 : 0
    Component 3 : 0
    Component 4 : 0
    Component 5 : 0
    Component 6 : 0
    Component 7 : 0
    Component 8 : 0
    Component 9 : 0
    
This perfectly matches with temporal explanation of the spoofed class.

**Spectral**

    Component 0 : 0
    Component 1 : 0
    Component 2 : 0
    Component 3 : 1
    Component 4 : 0
    Component 5 : 0
    Component 6 : 0
    Component 7 : 0
    
Clearcut match with spoofed class. These spoofed files do not exhibit the property that majority of genuine files exhibit.    
    